## 🌀 Quick Start Guide: It's all starting to unravel!

In this example we'll run through all the basic features the `unravelsports` package offers for converting a `kloppy` dataset of soccer tracking data into graphs for training binary classification graph neural networks using the `spektral` library.

This guide will go through the following steps:

- [**1. Process Data**](#1-processing-data). We'll show how to load a `kloppy` dataset and convert each individual frame into a single graph. All necessary steps (like setting the correct coordinate system, and left-right normalization) are done under the hood of the converter.
- [**1.1 Split Data**](#11-split-data).
- [**2. Compile Model**](#2-compile-model). We compile the built-in binary classification model as presented in [A Graph Neural Network Deep-dive into Successful Counterattacks {A. Sahasrabudhe & J. Bekkers}](https://github.com/USSoccerFederation/ussf_ssac_23_soccer_gnn).
- [**3. Fit Model**](#3-fit-model). Using the compiled model we fit it on the training set created in step [1.1 Splitting Data](#11-split-data).
- [**4. Evaluate Model Performance**](#4-evaluate-model-performance). We calculated model performance using the `metrics` passed into the compiled model.
- [**5. Predict**](#5-predict). Finally, we apply the trained model to unseen data.

<br>
<i>Before we get started it is important to note that the <b>unravelsports</b> library does not have built in functionality to create binary labels, these will need to be supplied by the reader. In this example we use the <b>dummy_labels()</b> functionality that comes with the package. This function creates a single binary label for each frame by randomly assigning it a 0 or 1 value.

When supplying your own labels they need to be in the form of a dictionary (more information on this can be found in the [in-depth Walkthrough](1_kloppy_gnn_train.ipynb)) </i>



-----


The first thing is to run `pip install unravelsports` if you haven't already!


In [1]:
%pip install unravelsports --quiet


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 1. Process Data

1. Load [Kloppy](https://github.com/PySport/kloppy) dataset. 
    See [in-depth Tutorial](1_kloppy_gnn_train.ipynb) on how do processes multiple match files, and to see an overview of all possible settings.
2. Convert to Graph format using `SoccerGraphConverter`
3. Create dataset for easy processing with [Spektral](https://graphneural.network/) using `GraphDataset`

In [ ]:
from unravel.soccer import SoccerGraphConverter, KloppyPolarsDataset
from unravel.utils import GraphDataset

from kloppy import sportec

# Load Kloppy dataset
kloppy_dataset = sportec.load_open_tracking_data(only_alive=True, limit=500)
kloppy_polars_dataset = KloppyPolarsDataset(
    kloppy_dataset=kloppy_dataset,
)
kloppy_polars_dataset.add_dummy_labels()
kloppy_polars_dataset.add_graph_ids(by=["frame_id"])

# Initialize the Graph Converter with dataset
# Here we use the default settings
converter = SoccerGraphConverter(dataset=kloppy_polars_dataset)

# Compute the graphs and add them to the GraphDataset
dataset = GraphDataset(graphs=converter.to_spektral_graphs())

#### 1.1 Split Data

Split the dataset with the built in `split_test_train_validation` method.

In [3]:
from spektral.data import DisjointLoader

train, test, val = dataset.split_test_train_validation(
    split_train=4, split_test=1, split_validation=1, random_seed=43
)

### 2. Compile Model

1. Initialize the `CrystalGraphClassifier`.
2. Compile the model with a loss function, optimizer and your preferred metrics.

Note: The model settings are chosen to reflect the model used in [A Graph Neural Network Deep-dive into Successful Counterattacks {A. Sahasrabudhe & J. Bekkers}](https://github.com/USSoccerFederation/ussf_ssac_23_soccer_gnn)

In [4]:
from unravel.classifiers import CrystalGraphClassifier

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, BinaryAccuracy

model = CrystalGraphClassifier()

model.compile(
    loss=BinaryCrossentropy(), optimizer=Adam(), metrics=[AUC(), BinaryAccuracy()]
)

### 3. Fit Model

1. Create a [`DisjointLoader`](https://graphneural.network/loaders/#disjointloader) for training and validation sets.
2. Fit the model. Note: set `use_multiprocessing=True` to speed up training significantly. 

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

batch_size = 32
epochs = 10

loader_tr = DisjointLoader(train, batch_size=batch_size)
loader_va = DisjointLoader(val, epochs=1, shuffle=False, batch_size=batch_size)

model.fit(
    loader_tr.load(),
    epochs=epochs,
    steps_per_epoch=loader_tr.steps_per_epoch,
    use_multiprocessing=True,
    validation_data=loader_va.load(),
    callbacks=[EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)],
)

Epoch 1/10


/Users/jbekkers/PycharmProjects/unravelsports/.venv311/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


11/11 [==============================] - 1s 16ms/step - loss: 21.7806 - auc: 0.5278 - binary_accuracy: 0.5419 - val_loss: 5.1682 - val_auc: 0.5000 - val_binary_accuracy: 0.5000
Epoch 2/10
 1/11 [=>............................] - ETA: 0s - loss: 9.2846 - auc: 0.3651 - binary_accuracy: 0.5000WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 3 batches). You may need to use the repeat() function when building your dataset.


11/11 [==============================] - 0s 6ms/step - loss: 4.5155 - auc: 0.5366 - binary_accuracy: 0.5449
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 2.0773 - auc: 0.4515 - binary_accuracy: 0.4731
Epoch 4/10
11/11 [==============================] - 0s 5ms/step - loss: 1.1006 - auc: 0.5205 - binary_accuracy: 0.5150
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 0.9159 - auc: 0.4915 - binary_accuracy: 0.5180
Epoch 6/10
11/11 [==============================] - 0s 5ms/step - loss: 0.8020 - auc: 0.4873 - binary_accuracy: 0.5060
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: 0.8067 - auc: 0.4960 - binary_accuracy: 0.5299
Epoch 8/10
11/11 [==============================] - 0s 6ms/step - loss: 0.7808 - auc: 0.5055 - binary_accuracy: 0.5299
Epoch 9/10
11/11 [==============================] - 0s 4ms/step - loss: 0.7661 - auc: 0.4937 - binary_accuracy: 0.5060
Epoch 10/10
11/11 [==============================] - 0s 5ms

### 4. Evaluate Model Performance

1. Create another `DisjointLoader`, this time for the test set.
2. Evaluate model performance on the test set. This evaluation function uses the `metrics` passed to `model.compile`

Note: Our performance is really bad because we're using random labels, very few epochs and a small dataset.

In [6]:
loader_te = DisjointLoader(test, epochs=1, shuffle=False, batch_size=batch_size)
results = model.evaluate(loader_te.load())

3/3 [==============================] - 0s 6ms/step - loss: 0.7001 - auc: 0.5000 - binary_accuracy: 0.4819


### 5. Predict

1. Use unseen data to predict on. In this example we're using the test dataset.
2. We have to re-create `loader_te` because `DisjointLoader` is a generator.
3. Setting `batch_size` and `use_multiprocessing=True` on prediction helps speed up the prediction

In [7]:
loader_te = DisjointLoader(test, batch_size=batch_size, epochs=1, shuffle=False)
loaded_pred = model.predict(loader_te.load(), use_multiprocessing=True)

3/3 [==============================] - 0s 5ms/step
